# Objective

1. Process the order data (zone-based,link-based,point-based, travel-time)


2. Initialize the driver data


3. Order dispatching


4. Repositioning location

In [9]:
import pandas as pd

from shapely.geometry import Point, Polygon

import geopandas as gp

import numpy as np

import random

import pulp

import folium

import networkx as nx


import math

In [10]:
def Check_zones(pnt,polys):
    
    key='None'
    
    for k, geom in polys.items():
        
        if pnt.within(geom):
            
            key=k
            
            break
            
    return key

def Check_links(pnt,zone,Zone_Link,Link_middle):
    
    Dic={}
    
    for link in Zone_Link[zone]:
        
        middle_pnt=Link_middle[link]
        
        Dic[link]=pnt.distance(middle_pnt)
        
    return min(Dic, key=Dic.get)

    
def Get_path(G,Points_link,Point_coordinate,source,target):
    
    link_path=list()
    
    try:
    
        path=nx.shortest_path(G, source=source, target=target,weight='weight')

        shortest_dis=nx.shortest_path_length(G, source=source, target=target,weight='weight')

        for i in range(1,len(path),1):

            pnts=path[i-1]+"&"+path[i]

            link_path.append(Points_link[pnts])
        
    except:
        
        shortest_dis=(Point_coordinate[source].distance(Point_coordinate[target])*111000)*1.3
    
    return link_path,shortest_dis

def Get_travel_time(dis,speed):
    
    return int(dis/speed)

def Get_matching_points(source,Point_coordinate,Points_list,radius):
    
    Point_candidates=list()
    
    for pnt in Points_list:
        
        dis=111000*(Point_coordinate[source].distance(Point_coordinate[pnt]))
    
        if dis<=radius:
            
            Point_candidates.append(pnt)
            
    return Point_candidates

def Compact_lists(arr):
    
    result=list()
    
    for a in arr:
        
        result=list(set(result+a))
        
    return result
            
def MILP_Optimization(Order,Driver,Utility):

    '''Define the problem'''

    model = pulp.LpProblem("Ride_Matching_Problems", pulp.LpMaximize)

    '''Construct our decision variable lists'''

    X = pulp.LpVariable.dicts("X",((i, j) for i in Order for j in Driver),lowBound=0,upBound=1,cat='Integer')

    '''Objective Function'''

    model += (pulp.lpSum([Utility[i][j] * X[(i, j)] for i in Order for j in Driver]))


    '''Each driver can only serve one order'''

    for i in Order:

        model += pulp.lpSum([X[(i, j)] for j in Driver]) <=1

    '''Each order can only be assigned one driver'''

    for j in Driver:

         model += pulp.lpSum([X[(i, j)] for i in Order]) <=1



    model.solve()

    result={}

    for var in X:

        var_value = X[var].varValue
        
        if var_value !=0:
            
            result[var[0]]=var[1]
    

    return result   
    
    

In [11]:
'''Param'''

s_sec=25200

e_sec=36000

Start_step=2520

End_step=3600

Max_waiting=30

radius=1000

speed=10 # 10 m/seconds

Driver_num=3000





In [12]:
'''Load data'''

'''Zone-related data'''

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Link-related data'''

Link_middle=np.load('./Data/NYC_Zones/Link_middle.npy',allow_pickle=True).item()

Link_Point=np.load('./Data/NYC_Zones/Link_Point.npy',allow_pickle=True).item()

'''Point-related data'''

Points_list=np.load('./Data/NYC_Zones/Points_list.npy',allow_pickle=True)

Points_link=np.load('./Data/NYC_Zones/Points_link.npy',allow_pickle=True).item()

Point_coordinate=np.load('./Data/NYC_Zones/Point_coordinate.npy',allow_pickle=True).item()

'''GeoSeries Object'''

polys = gp.GeoSeries(Taxi_Zones)

'''Road network Object'''

G = nx.Graph()

G.add_nodes_from(Points_list)

G.add_weighted_edges_from(list(Link_Point.values()))


# Don't run this code frequently !

In [24]:
'''1. Processing the Order data'''

'''Link based and point based'''

Order_df=pd.read_csv('./Data/NewYork_New/pick_day_1.csv',header=None,names=['Pick_Second','Trip_Duration',\
                                                     'Pickup_Latitude','Pickup_Longitude',\
                                                     'Dropoff_Latitude','Dropoff_Longitude',\
                                                     'Trip_Distance','Euclidean','Manhattan'])

Order_df['Pickup_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),polys),axis=1)

Order_df['Dropoff_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),polys),axis=1)

Order_df=Order_df.loc[(Order_df['Pickup_Zone']!='None')&(Order_df['Dropoff_Zone']!='None')&(Order_df['Pick_Second']>=s_sec)&(Order_df['Pick_Second']<e_sec)]

Order_df=Order_df.sort_values(by=['Pick_Second'])

Order_df=Order_df.rename(columns={'Pick_Second':'Arrive_Second'})

Order_df=Order_df.reset_index(drop=True)

Order_df['Order_id']=['O'+str(i) for i in Order_df.index]

Order_df['Driver_id']='Waiting'

Order_df['Arrive_step']=Order_df.apply(lambda x:math.ceil(x['Arrive_Second']/10),axis=1)

Order_df=Order_df[['Order_id','Driver_id','Arrive_step',\
                   'Pickup_Latitude','Pickup_Longitude',\
                   'Dropoff_Latitude','Dropoff_Longitude',\
                    'Pickup_Zone','Dropoff_Zone']]

Order_df['Pickup_Link']=Order_df.apply(lambda x:Check_links(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),x['Pickup_Zone'],Zone_Link,Link_middle),axis=1)

Order_df['Dropoff_link']=Order_df.apply(lambda x:Check_links(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),x['Dropoff_Zone'],Zone_Link,Link_middle),axis=1)

Order_df['Pickup_Point']=Order_df.apply(lambda x:Link_Point[x['Pickup_Link']][0],axis=1)

Order_df['Dropoff_Point']=Order_df.apply(lambda x:Link_Point[x['Dropoff_link']][1],axis=1)

Order_df['Travel_dis']=Order_df.apply(lambda x:Get_path(G,Points_link,Point_coordinate,x['Pickup_Point'],x['Dropoff_Point'])[1],axis=1)

Order_df['Travel_time']=Order_df.apply(lambda x:Get_travel_time(x['Travel_dis'],speed),axis=1)

# Order_df.to_csv('./Data/NYC_Trips/Order_df.csv')

Order_df



In [27]:
'''2. Initialize the driver data'''

Driver_list=['D'+str(i) for i in range(Driver_num)]

Driver_df=pd.DataFrame(Driver_list,columns=['Driver_id'])

'''Initiial zones'''

Invalid_zones=['Zone_19','Zone_20','Zone_21']

Valid_zones=list(set(Zone_list).difference(set(Invalid_zones)))

Driver_df['Zone']=Driver_df.apply(lambda x:random.choice(Valid_zones),axis=1)

Driver_df['Latitude']=Driver_df.apply(lambda x:Zone_Center[x['Zone']][1],axis=1)

Driver_df['Longitude']=Driver_df.apply(lambda x:Zone_Center[x['Zone']][0],axis=1)

Driver_df['Link']=Driver_df.apply(lambda x:Check_links(Point(x['Longitude'],x['Latitude']),x['Zone'],Zone_Link,Link_middle),axis=1)

Driver_df['Point']=Driver_df.apply(lambda x:Link_Point[x['Link']][0],axis=1)

Driver_df['second']=s_sec

Driver_df['Step']=Driver_df.apply(lambda x:math.ceil(x['second']/10),axis=1)

Driver_df['Order_id']='Idle'

Driver_df=Driver_df[['Driver_id','Order_id','Step',\
                   'Latitude','Longitude',\
                   'Zone','Link','Point']]

# Driver_df.to_csv('./Data/NYC_Trips/Driver_df.csv')

Driver_df



,Driver_id,Order_id,Step,Latitude,Longitude,Zone,Link,Point
0,D0,Idle,2520,40.714733,-73.983025,Zone_56,Link_9162,Point_5891
1,D1,Idle,2520,40.731821,-73.976598,Zone_51,Link_653,Point_5494
2,D2,Idle,2520,40.766948,-73.959635,Zone_32,Link_4984,Point_4687
3,D3,Idle,2520,40.756688,-73.972356,Zone_41,Link_5110,Point_2346
4,D4,Idle,2520,40.762253,-73.989845,Zone_8,Link_3535,Point_2394
5,D5,Idle,2520,40.804334,-73.951292,Zone_4,Link_8223,Point_1390
6,D6,Idle,2520,40.761900,-73.949952,Zone_48,Link_9382,Point_2142
7,D7,Idle,2520,40.712038,-74.016079,Zone_3,Link_567,Point_267
8,D8,Idle,2520,40.782478,-73.965554,Zone_7,Link_8589,Point_3408
9,D9,Idle,2520,40.748428,-73.999917,Zone_10,Link_5645,Point_5438


In [13]:
'''Dispatching Procedure'''

Order_df=pd.read_csv('./Data/NYC_Trips/Order_df.csv')

Order_df=Order_df.drop(columns=['Unnamed: 0'])

Order_df['Pickup_step']=End_step

Order_df=Order_df[['Order_id','Driver_id','Arrive_step','Pickup_step',\
                   'Pickup_Latitude','Pickup_Longitude',\
                   'Dropoff_Latitude','Dropoff_Longitude',\
                    'Pickup_Zone','Dropoff_Zone',\
                   'Pickup_Link','Dropoff_link',\
                   'Pickup_Point','Dropoff_Point',\
                   'Travel_dis','Travel_time']]

Driver_df=pd.read_csv('./Data/NYC_Trips/Driver_df.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Quatitive_results=list()


for step in range(Start_step,End_step,1):
    
    
    '''(1) select unifinished orders'''
    
    Order_batch=Order_df[(Order_df['Arrive_step']<=step)&(Order_df['Driver_id']=='Waiting')]

    Order_arr=list();
    
    Travel_time={}
    
    Order_info={}
    
    Match_Points={};
    
    for idx,row in Order_batch.iterrows():

        order_id=row['Order_id']

        Order_arr.append(order_id)
        
        Travel_time[order_id]=int(row['Travel_time']/10)
        
        Order_info[order_id]={'Pickup_Point':row['Pickup_Point'],\
                              'Dropoff_Latitude':row['Dropoff_Latitude'],\
                              'Dropoff_Longitude':row['Dropoff_Longitude'],\
                              'Dropoff_Zone':row['Dropoff_Zone'],\
                              'Dropoff_link':row['Dropoff_link'],\
                              'Dropoff_Point':row['Dropoff_Point']}
        
        Match_Points[order_id]=Get_matching_points(row['Pickup_Point'],Point_coordinate,Points_list,radius)
        
    Operation_Points=Compact_lists(list(Match_Points.values()))
    
    '''(2) select Idle drivers'''
    
    Idle_drivers=list(Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle'),'Driver_id'])
    
    Driver_batch=Driver_df[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle')&(Driver_df['Point'].isin(Operation_Points))]
    
    Driver_arr=list()

    Driver_Points={}
    
    Points_Drivers={}
    
    for idx,row in Driver_batch.iterrows():
        
        driver_id=row['Driver_id']

        Driver_arr.append(driver_id)

        Driver_Points[driver_id]=row['Point']
        
        if row['Point'] in Points_Drivers.keys():
            
            Points_Drivers[row['Point']].append(driver_id)
            
        else:
            
            Points_Drivers[row['Point']]=[driver_id]
            
            
    '''(3) Construct Matching Utility'''
    
    Utility_matrix={}

    for order_id in Order_arr:

        Utility_matrix[order_id]={}
        
        origin=Order_info[order_id]['Pickup_Point']
        
        match_points=Match_Points[order_id]
        
        for point in match_points:
            
            if point in Points_Drivers.keys():
                
                for driver_id in Points_Drivers[point]:
                    
                    pickup_dis=Get_path(G,Points_link,Point_coordinate,origin,point)[1]
                    
                    if pickup_dis!=0.0:
                        
                        Utility_matrix[order_id][driver_id]=1/(pickup_dis)
                        
                    else:
                        
                        Utility_matrix[order_id][driver_id]=99.0
                        
        for driver_id in Driver_arr:

            if driver_id not in Utility_matrix[order_id].keys():

                Utility_matrix[order_id][driver_id]=0.0   
                
    '''(4) Optimize matching results '''
    
    Matching_result=MILP_Optimization(Order_arr,Driver_arr,Utility_matrix)
    
    print('Current step ',step,'Waiting orders: ',len(Order_arr),'Selected drivers: ',len(Driver_arr),'Matching pairs: ',len(Matching_result))
    
    Quatitive_results.append([step,len(Order_arr),len(Idle_drivers),len(Matching_result)])

    print('*'*50)
    
    
    '''(5) Update results of orders'''
    
    '''(5-1) Update the matched orders and drivers'''
    
    for order_id,driver_id in Matching_result.items():
        
        '''Matched order'''
        
        Order_df.loc[(Order_df['Arrive_step']<=step)&(Order_df['Order_id']==order_id),'Pickup_step']=step
        
        Order_df.loc[(Order_df['Arrive_step']<=step)&(Order_df['Order_id']==order_id),'Driver_id']=driver_id
        
        '''Matched driver'''

        Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Driver_id']==driver_id)&(Driver_df['Point'].isin(Operation_Points)),'Order_id']=order_id
        
        Added_item={'Driver_id': driver_id,\
                    'Order_id':'Idle',\
                    'Step':step+Travel_time[order_id],\
                    'Latitude':Order_info[order_id]['Dropoff_Latitude'],\
                    'Longitude':Order_info[order_id]['Dropoff_Longitude'],\
                    'Zone':Order_info[order_id]['Dropoff_Zone'],\
                    'Link':Order_info[order_id]['Dropoff_link'],\
                    'Point':Order_info[order_id]['Dropoff_Point']}
        
        Driver_df=Driver_df.append(Added_item, ignore_index=True)
        
    '''(5-2) Update the unmatched orders'''
    
    Matched_orders=list(Matching_result.keys())
    
    Unmatched_orders=[O for O in Order_arr if O not in Matched_orders]
    
    if len(Unmatched_orders)!=0:
        
        Order_df.loc[(step-Order_df['Arrive_step']>Max_waiting)&(Order_df['Order_id'].isin(Unmatched_orders)),'Driver_id']='Cancelled'
        
        
    '''(5-3) Update the unmatehed drivers: Repositioning'''
    
    Idle_drivers=list(Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Order_id']=='Idle'),'Driver_id'])
    
    Next_Driver_df=Driver_df.loc[(Driver_df['Step']==step)&(Driver_df['Driver_id'].isin(Idle_drivers))]
    
    '''Just waiting at the current location'''
    
    Next_Driver_df['Step']=Next_Driver_df.apply(lambda x:x['Step']+1,axis=1)
    
    Driver_df=pd.concat([Driver_df,Next_Driver_df],ignore_index=True)
    
    if step >2600:
        
        break
    

Current step  2520 Waiting orders:  1 Selected drivers:  296 Matching pairs:  1
**************************************************


/Users/yueyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Current step  2521 Waiting orders:  7 Selected drivers:  684 Matching pairs:  5
**************************************************
Current step  2522 Waiting orders:  8 Selected drivers:  400 Matching pairs:  4
**************************************************
Current step  2523 Waiting orders:  14 Selected drivers:  1097 Matching pairs:  8
**************************************************
Current step  2524 Waiting orders:  13 Selected drivers:  345 Matching pairs:  5
**************************************************
Current step  2525 Waiting orders:  16 Selected drivers:  860 Matching pairs:  7
**************************************************
Current step  2526 Waiting orders:  19 Selected drivers:  969 Matching pairs:  9
**************************************************
Current step  2527 Waiting orders:  16 Selected drivers:  927 Matching pairs:  6
**************************************************
Current step  2528 Waiting orders:  19 Selected drivers:  872 Matching pairs:

Current step  2583 Waiting orders:  32 Selected drivers:  1075 Matching pairs:  12
**************************************************
Current step  2584 Waiting orders:  24 Selected drivers:  647 Matching pairs:  5
**************************************************
Current step  2585 Waiting orders:  28 Selected drivers:  638 Matching pairs:  9
**************************************************
Current step  2586 Waiting orders:  29 Selected drivers:  811 Matching pairs:  12
**************************************************
Current step  2587 Waiting orders:  31 Selected drivers:  1458 Matching pairs:  15
**************************************************
Current step  2588 Waiting orders:  21 Selected drivers:  410 Matching pairs:  5
**************************************************
Current step  2589 Waiting orders:  24 Selected drivers:  672 Matching pairs:  8
**************************************************
Current step  2590 Waiting orders:  30 Selected drivers:  1317 Matching